In [6]:
# Notebook settings
import logging
from importlib import reload
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


logging.basicConfig(
    level=logging.DEBUG, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [7]:
# Default
import pandas as pd
import os
import sys

module_path = os.path.abspath(
    "/mnt/cmnfs/proj/ORIGINS/protMSD/maxquant/ScanByScan/")
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
# path to MaxQuant results (evidence.txt) of the same RAW file, used for constructing reference dictionary
maxquant_file_dict = (
    "/cmnfs/proj/ORIGINS/data/ecoli/ss/DDA/MQ/combined/txt/"
    "evidence_1_1_FilteredByClosestRT_transfer_RT_pred_filtered_withIso.pkl"
)
maxquant_file_exp = (
    "/cmnfs/proj/ORIGINS/data/ecoli/ss/DDA/MQ/combined/txt/evidence_1.txt"
)
maxquant_file_dict_expRT = maxquant_file_dict[:-4] + "_expRTrange.pkl"
print(maxquant_file_dict_expRT)

/cmnfs/proj/ORIGINS/data/ecoli/ss/DDA/MQ/combined/txt/evidence_1_1_FilteredByClosestRT_transfer_RT_pred_filtered_withIso_expRTrange.pkl


In [9]:
# Load maxquant result (reference)
Maxquant_result_dict = pd.read_pickle(filepath_or_buffer=maxquant_file_dict)
Maxquant_result_exp = pd.read_csv(
    filepath_or_buffer=maxquant_file_exp, sep="\t")

In [10]:
# prepare maxquant exp result
Maxquant_result_exp_for_merge = Maxquant_result_exp[
    [
        "Modified sequence",
        "Charge",
        "Retention time",
        "Calibrated retention time start",
        "Calibrated retention time finish",
    ]
]
Maxquant_result_exp_for_merge = (
    Maxquant_result_exp_for_merge.groupby(["Modified sequence", "Charge"])
    .agg(
        {
            "Retention time": "median",
            "Calibrated retention time start": "median",
            "Calibrated retention time finish": "median",
        }
    )
    .reset_index()
)

In [11]:
import numpy as np

a = np.array([None, None, None])
a[0] is None

True

In [12]:
Maxquant_result_dict_exp_RT = pd.merge(
    left=Maxquant_result_dict,
    right=Maxquant_result_exp_for_merge,
    on=["Modified sequence", "Charge"],
    suffixes=[None, "_exp"],
    how="left",
)
# Merge retention time result: if exp RT is not available, use predicted RT,
# start time and end time from Maxquant exp and if not available from reference run
Maxquant_result_dict_exp_RT["Retention time new"] = Maxquant_result_dict_exp_RT[
    "Retention time_exp"
].fillna(Maxquant_result_dict_exp_RT["predicted_RT"])
Maxquant_result_dict_exp_RT["Retention time start"] = Maxquant_result_dict_exp_RT[
    "Calibrated retention time start_exp"
].fillna(Maxquant_result_dict_exp_RT["Calibrated retention time start"])
Maxquant_result_dict_exp_RT["Retention time end"] = Maxquant_result_dict_exp_RT[
    "Calibrated retention time finish_exp"
].fillna(Maxquant_result_dict_exp_RT["Calibrated retention time finish"])
Maxquant_result_dict_exp_RT.to_pickle(maxquant_file_dict_expRT)